githubからデータを取得

In [ ]:
!git clone https://github.com/Apress/mastering-ml-w-python-in-six-steps.git
!ln -s mastering-ml-w-python-in-six-steps/Chapter_3_Code/Code/Data Data

warningを非表示

In [ ]:
import warnings
warnings.filterwarnings('ignore')

3-1 ダミー変数の作成

In [ ]:
import pandas as pd
from patsy import dmatrices
df = pd.DataFrame({'A': ['high', 'medium', 'low'],
                   'B': [10,20,30]},
                    index=[0, 1, 2])
                    
# pandasのget_dummies関数を使う
df_with_dummies= pd.get_dummies(df, prefix='A', columns=['A'])
print (df_with_dummies)

3-2 カテゴリカル変数の数値への変換

In [ ]:
# pandasのfactorize関数を使う
df['A_pd_factorized'] = pd.factorize(df['A'])[0]

# sklearnのLabelEncoder関数を代わりに使うこともできる
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['A_LabelEncoded'] = le.fit_transform(df.A)
print (df)

3-3. 正規化とスケーリング

In [ ]:
from sklearn import datasets
import numpy as np
from sklearn import preprocessing
iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
std_scale = preprocessing.StandardScaler().fit(X) #標準化
X_std = std_scale.transform(X)
minmax_scale = preprocessing.MinMaxScaler().fit(X) #正規化
X_minmax = minmax_scale.transform(X)
print('Mean before standardization: petal length={:.1f}, petal width={:.1f}'
      .format(X[:,0].mean(), X[:,1].mean()))
print('SD before standardization: petal length={:.1f}, petal width={:.1f}'
      .format(X[:,0].std(), X[:,1].std()))
print('Mean after standardization: petal length={:.1f}, petal width={:.1f}'
      .format(X_std[:,0].mean(), X_std[:,1].mean()))
print('SD after standardization: petal length={:.1f}, petal width={:.1f}'
      .format(X_std[:,0].std(), X_std[:,1].std()))
print('\nMin value before min-max scaling: patel length={:.1f}, patel width={:.1f}'
      .format(X[:,0].min(), X[:,1].min()))
print('Max value before min-max scaling: petal length={:.1f}, petal width={:.1f}'
      .format(X[:,0].max(), X[:,1].max()))
print('Min value after min-max scaling: patel length={:.1f}, patel width={:.1f}'
      .format(X_minmax[:,0].min(), X_minmax[:,1].min()))
print('Max value after min-max scaling: petal length={:.1f}, petal width={:.1f}'
      .format(X_minmax[:,0].max(), X_minmax[:,1].max()))


3-4. 単変量解析

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
iris = datasets.load_iris()

# Dataframeの変換
iris = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['species'])

# クラスラベルの置き換え
iris.species = np.where(iris.species == 0.0, 'setosa', np.where(iris.species==1.0,'versicolor', 'virginica'))

# 列名からスペースを削除
iris.columns = iris.columns.str.replace(' ','')
iris.describe()

In [ ]:
print (iris['species'].value_counts())

3-5 Pnadas DataFrameの可視化

In [ ]:
# プロットのサイズを設定
plt.figure(figsize=(15,8))
iris.hist()        # ヒストグラム(histobram)のプロット
plt.suptitle("Histogram", fontsize=12) # ヒストグラムにサブタイトルをつける
plt.tight_layout(pad=1)
plt.show()
iris.boxplot()     # 箱ひげ図のプロット
plt.title("Bar Plot", fontsize=16)
plt.tight_layout(pad=1)
plt.show()


3-6 多変量解析

In [ ]:
# 各列の平均値を種ごとに表示
iris.groupby(by = "species").mean()

# 各ラベルクラスに対する各特徴の平均値をプロット
iris.groupby(by = "species").mean().plot(kind="bar")
plt.title('Class vs Measurements')
plt.ylabel('mean measurement(cm)')
plt.xticks(rotation=0)  # x軸の方向を設定
plt.grid(True)

# bbox_to_anchorオプションを使用してプロット領域の外側に凡例を配置
plt.legend(loc="upper left", bbox_to_anchor=(1,1))

3-7相関行列

In [ ]:
corr = iris.corr()
print(corr)
import statsmodels.api as sm
sm.graphics.plot_corr(corr, xnames=list(corr.columns))
plt.show()

3-8 散布図行列

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(iris, figsize=(10, 10))
# サブタイトルを散布図につける
plt.suptitle("Pair Plot", fontsize=20)

3-9 家庭での勉強時間と生徒の成績

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# データの読み込み
df = pd.read_csv('Data/Grade_Set_1.csv')
print(df)

# 散布図のプロット
df.plot(kind='scatter', x='Hours_Studied', y='Test_Grade', title='Grade vs Hours Studied')
plt.show()

# 各変数の相関の確認
print("Correlation Matrix: ")
print(df.corr())

3-10　線形回帰

In [ ]:
import sklearn.linear_model as lm

# 線形回帰のためのインスタンスを生成する
lr = lm.LinearRegression()
x= df.Hours_Studied[:, np.newaxis] # 独立変数
y= df.Test_Grade.values            # 従属変数

# 訓練データセットを使ってモデルを訓練
lr.fit(x, y)
print("Intercept: ", lr.intercept_)
print("Coefficient: ", lr.coef_)

# 与えられたｘに対して手動で予測を行う
print("Manual prediction :", lr.intercept_ + lr.coef_ * 6)

# ライブラリに含まれる関数を使って予測を行う
print("Using predict function: ", lr.predict([[6]]))

# フィッティングした直線をプロットする
plt.scatter(x, y,  color='black')
plt.plot(x, lr.predict(x), color='blue', linewidth=3)
plt.title('Grade vs Hours Studied')
plt.ylabel('Test_Grade')
plt.xlabel('Hours_Studied')


3-11 線形回帰モデルの精度行列

In [ ]:
from sklearn.metrics import r2_score , mean_absolute_error, mean_squared_error

# Dataframeに予測値を追加
df['Test_Grade_Pred'] = lr.predict(x)

# 決定係数を手動で計算
df['SST'] = np.square(df['Test_Grade'] - df['Test_Grade'].mean())
df['SSR'] = np.square(df['Test_Grade_Pred'] - df['Test_Grade'].mean())
print("Sum of SSR:", df['SSR'].sum())
print("Sum of SST:", df['SST'].sum())
print(df)
df.to_csv('r-squared.csv', index=False)
print("R Squared using manual calculation: ", df['SSR'].sum() / df['SST'].sum())

# ライブラリの関数を使用
print("R Squared using built-in function: ", r2_score(df.Test_Grade,  df.Test_Grade_Pred))
print("Mean Absolute Error: ", mean_absolute_error(df.Test_Grade, df.Test_Grade_Pred))
print("Root Mean Squared Error: ", np.sqrt(mean_squared_error(df.Test_Grade, df.Test_Grade_Pred)))

3-12 外れ値 vs. 決定係数

In [ ]:
# データの読み込み
df = pd.read_csv('Data/Grade_Set_1.csv')
df.loc[9] = np.array([5, 100])
x= df.Hours_Studied[:, np.newaxis] # 独立変数
y= df.Test_Grade.values            # 従属変数

# トレーニングデータセットをつかってモデルをトレーニング
lr.fit(x, y)
print("Intercept: ", lr.intercept_)
print("Coefficient: ", lr.coef_)

# 与えられたxに対して手動で予測
print("Manual prediction :", 54.4022988505747 + 4.64367816*6)

# ライブラリを使って予測
print("Using predict function: ", lr.predict([[6]]))

# 直線をプロット
plt.scatter(x, y,  color='black')
plt.plot(x, lr.predict(x), color='blue', linewidth=3)
plt.title('Grade vs Hours Studied')
plt.ylabel('Test_Grade')
plt.xlabel('Hours_Studied')

# Dataframeに予測値を追加
df['Test_Grade_Pred'] = lr.predict(x)

# ライブラリの関数を使用
print("R Squared : ", r2_score(df.Test_Grade,  df.Test_Grade_Pred))
print("Mean Absolute Error: ", mean_absolute_error(df.Test_Grade, 
df.Test_Grade_Pred))
print("Root Mean Squared Error: ", np.sqrt(mean_squared_error(df.Test_Grade, df.Test_Grade_Pred)))


3-13.  多項式回帰

In [ ]:
x = np.linspace(-3,3,1000) # -3から3の範囲でサンプリングされた1000個の数
# プロット
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3)
ax1.plot(x, x)
ax1.set_title('linear')
ax2.plot(x, x**2)
ax2.set_title('degree 2')
ax3.plot(x, x**3)
ax3.set_title('degree 3')
ax4.plot(x, x**4)
ax4.set_title('degree 4')
ax5.plot(x, x**5)
ax5.set_title('degree 5')
ax6.plot(x, x**6)
ax6.set_title('degree 6')
plt.tight_layout()# レイアウトをつめて出力

3-14. 多項式回帰の例

In [ ]:
# 線形回帰の関数をインポート
import sklearn.linear_model as lm
# データを読み込み
df = pd.read_csv('Data/Grade_Set_2.csv')
print(df)
# 散布図をプロット
df.plot(kind='scatter', x='Hours_Studied', y='Test_Grade', title='Grade vs Hours Studied')
#変数間の相関関係を確認
print("Correlation Matrix: ")
print(df.corr())
# 線形回帰オブジェクトの作成
lr = lm.LinearRegression()
x= df.Hours_Studied[:, np.newaxis]           # 独立変数
y= df.Test_Grade                             # 従属変数
# トレーニングセットを使ったモデルのトレーニング
lr.fit(x, y)
# フィッティングした線をプロット
plt.scatter(x, y,  color='black')
plt.plot(x, lr.predict(x), color='blue', linewidth=3)
plt.title('Grade vs Hours Studied')
plt.ylabel('Test_Grade')
plt.xlabel('Hours_Studied')
print("R Squared: ", r2_score(y, lr.predict(x)))


3-15 多項式の次数が異なる場合の決定係数

In [ ]:
lr = lm.LinearRegression()
x= df.Hours_Studied        # 独立変数
y= df.Test_Grade           # 従属変数
# NumPyのvander関数は入力ベクトルの累乗を返す
for deg in [1, 2, 3, 4, 5]:
    lr.fit(np.vander(x, deg + 1), y)
    y_lr = lr.predict(np.vander(x, deg + 1))
    plt.plot(x, y_lr, label='degree ' + str(deg))
    plt.legend(loc=2)
    print("R-squared for degree " + str(deg) + " = ",  r2_score(y, y_lr))
plt.plot(x, y, 'ok')


3-16. Scikit-learnを使った多項式回帰

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

x= df.Hours_Studied[:, np.newaxis] # 独立変数
y= df.Test_Grade                   # 従属変数
degree = 3
model = make_pipeline(PolynomialFeatures(degree), lr)
model.fit(x, y)
plt.scatter(x, y,  color='black')
plt.plot(x, model.predict(x), color='green')
plt.title('Grade vs Hours Studied')
plt.ylabel('Test_Grade')
plt.xlabel('Hours_Studied')
print("R Squared using built-in function: ", r2_score(y, model.predict(x)))

3-17. 多重共線性と分散拡大係数

In [ ]:
# データの読み込み
df = pd.read_csv('Data/Housing_Modified.csv')

# バイナリを数値化されたブール値に変換
lb = preprocessing.LabelBinarizer()
df.driveway = lb.fit_transform(df.driveway)
df.recroom = lb.fit_transform(df.recroom)
df.fullbase = lb.fit_transform(df.fullbase)
df.gashw = lb.fit_transform(df.gashw)
df.airco = lb.fit_transform(df.airco)
df.prefarea = lb.fit_transform(df.prefarea)

# ダミー変数の作成
df_stories = pd.get_dummies(df['stories'], prefix='stories', drop_first=True)

# ダミー変数をメインのデータフレームに結合する
df = pd.concat([df, df_stories], axis=1)
del df['stories']

# statmodels graphicsパッケージのplot_corを使って相関行列をプロット
# 相関行列の作成
corr = df.corr()
sm.graphics.plot_corr(corr, xnames=list(corr.columns))
plt.show()


3-18. 多重共線性の除去

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor, OLSInfluence
# 機能名のリストを作成
independent_variables = ['lotsize', 'bedrooms', 'bathrms','driveway', 'recroom',
                         'fullbase','gashw','airco','garagepl', 'prefarea',
                         'stories_one','stories_two','stories_three']
                         
# リストを使って元のDataFrameからサブセットを選択
X = df[independent_variables]
y = df['price']
thresh = 10
for i in np.arange(0,len(independent_variables)):
    vif = [variance_inflation_factor(X[independent_variables].values, ix) for ix in range(X[independent_variables].shape[1])]
    maxloc = vif.index(max(vif))
    if max(vif) > thresh:
        print("vif :", vif)
        print("dropping " + X[independent_variables].columns[maxloc] + " at index: " + str(maxloc))
        del independent_variables[maxloc]
    else:
        break
print('Final variables:', independent_variables)


3-19.  多変量線形回帰モデルの構築

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
# 機能名のリストを作成
independent_variables = [
'lotsize', 'bathrms','driveway', 'fullbase','gashw', 'airco','garagepl',
                         'prefarea','stories_one','stories_three']

#リストを使って元のDataFrameからサブセットを選択
X = df[independent_variables]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.80, random_state=1)

# フィッティングしたモデルを作成
lm = sm.OLS(y_train, X_train).fit()

# 要約を表示
print(lm.summary())

# テストモデルを使って予測
y_train_pred = lm.predict(X_train)
y_test_pred = lm.predict(X_test)
y_pred = lm.predict(X) # full data
print("Train MAE: ", metrics.mean_absolute_error(y_train, y_train_pred))
print("Train RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Test MAE: ", metrics.mean_absolute_error(y_test, y_test_pred))
print("Test RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))


3-20. 正規化された残差とレバレッジのプロット

In [ ]:
from statsmodels.graphics.regressionplots import plot_leverage_resid2
fig, ax = plt.subplots(figsize=(8,6))
fig = plot_leverage_resid2(lm, ax = ax)

3-21. 外れ値の探索

In [ ]:
# 外れ値の探索
# Bonferroni法による探索
test = lm.outlier_test()
print('Bad data points (bonf(p) < 0.05):')
print(test[test['bonf(p)'] < 0.05])

3-22. 等分散性と正規性

In [ ]:
# 等分散性をチェックするためのグラフ
plt.plot(lm.resid,'o')
plt.title('Residual Plot')
plt.ylabel('Residual')
plt.xlabel('Observation Numbers')
plt.show()
plt.hist(lm.resid, density=True)

3-23. 線型性のチェック

In [ ]:
# 直線のプロット
fig = plt.figure(figsize=(10,15))
fig = sm.graphics.plot_partregress_grid(lm, fig=fig)

3-24. 正則化

In [ ]:
from sklearn import linear_model

# データの読み込み
df = pd.read_csv('Data/Grade_Set_2.csv')
df.columns = ['x','y']
for i in range(2,50):               # 1乗は処理の必要なし
    colname = 'x_%d'%i              # 列名は「x_べき数」となる
    df[colname] = df['x']**i
independent_variables = list(df.columns)
independent_variables.remove('y')
X= df[independent_variables]       # 独立変数
y= df.y                            # 従属変数

# 学習データとテストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.80, random_state=1)

# リッジ回帰
lr = linear_model.Ridge(alpha=0.001)
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)
print("------ Ridge Regression ------")
print("Train MAE: ", metrics.mean_absolute_error(y_train, y_train_pred))
print("Train RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, 
y_train_pred)))
print("Test MAE: ", metrics.mean_absolute_error(y_test, y_test_pred))
print("Test RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, 
y_test_pred)))
print("Ridge Coef: ", lr.coef_)

# ラッソ回帰
lr = linear_model.Lasso(alpha=0.001)
lr.fit(X_train, y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)
print("----- LASSO Regression -----")
print("Train MAE: ", metrics.mean_absolute_error(y_train, y_train_pred))
print("Train RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Test MAE: ", metrics.mean_absolute_error(y_test, y_test_pred))
print("Test RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
print("LASSO Coef: ", lr.coef_)

3-25. 非線形回帰

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

%matplotlib inline

x= np.array([-2,-1.64,-0.7,0,0.45,1.2,1.64,2.32,2.9])
y = np.array([1.0, 1.5, 2.4, 2, 1.49, 1.2, 1.3, 1.2, 0.5])

# 非線形曲線へのフィッティング
def func(x, p1,p2):
    return p1*np.sin(p2*x) + p2*np.cos(p1*x)

popt, pcov = curve_fit(func, x, y, p0=(1.0,0.2))
p1 = popt[0]
p2 = popt[1]
residuals = y - func(x,p1,p2)
fres = sum(residuals**2)
curvex=np.linspace(-2,3,100)
curvey=func(curvex,p1,p2)

plt.plot(x,y,'bo ')
plt.plot(curvex,curvey,'r')
plt.title('Non-linear fitting')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['data','fit'],loc='best')
plt.show()


3-26 ロジスティック回帰

In [ ]:
import sklearn.linear_model as lm

# データの読み込み
df = pd.read_csv('Data/Grade_Set_1_Classification.csv')
x= df.Hours_Studied[:, np.newaxis] # 独立変数
y= df.Result                       # 従属変数

# 線形回帰オブジェクトの作成
lr = lm.LinearRegression()

# 訓練データセットを使ってモデルを訓練
lr.fit(x, y)

# 直線のプロット
plt.scatter(x, y,  color='black')
plt.plot(x, lr.predict(x), color='blue', linewidth=3)
plt.title('Hours Studied vs Result')
plt.ylabel('Result')
plt.xlabel('Hours_Studied')

# Dataframeへ予測値の追加
df['Result_Pred'] = lr.predict(x)

# 標準ライブラリを使用
print ("R Squared : ", r2_score(df.Result, df.Result_Pred))
print ("Mean Absolute Error: ", mean_absolute_error(df.Result, df.Result_Pred))
print ("Root Mean Squared Error: ", np.sqrt(mean_squared_error(df.Result, df.Result_Pred)))


3-27. シグモイド関数のプロット

In [ ]:
# シグモイド関数のプロット
x = np.linspace(-10, 10, 100)
y = 1.0 / (1.0 + np.exp(-x))
plt.plot(x, y, 'r-', label='logit')
plt.legend(loc='lower right')

3-28. Scikit-learnを使ったロジスティック回帰

In [ ]:
from sklearn.linear_model import LogisticRegression

# 手動でインターセプトを追加
df['intercept'] = 1
independent_variables = ['Hours_Studied', 'intercept']
x = df[independent_variables]         # 独立変数
y = df['Result']                      # 従属変数

# ロジスティック回帰モデルをインスタンス化して，XとYにフィットさせる
model = LogisticRegression(max_iter=10)
model = model.fit(x, y)

# 訓練データセットでの精度確認
model.score(x, y)

# predict_probaでy = 0とy = 1の確率を含む配列を返す
print ('Predicted probability:', model.predict_proba(x)[:,1])

# predictでは確率(y=1)の値が0.5を超えた場合1，超えない場合は0に変換する
print ('Predicted Class:',model.predict(x))

# フィッティングした線のプロット
plt.scatter(df.Hours_Studied, y,  color='black')
plt.yticks([0.0, 0.5, 1.0])
plt.plot(df.Hours_Studied, model.predict_proba(x)[:,1], color='blue', linewidth=3)
plt.title('Hours Studied vs Result')
plt.ylabel('Result')
plt.xlabel('Hours_Studied')
plt.show()


3-29. 混同行列

In [ ]:
from sklearn import metrics

# 評価指標の作成
print ("Accuracy :", metrics.accuracy_score(y, model.predict(x)))
print ("AUC :", metrics.roc_auc_score(y, model.predict_proba(x)[:,1]))
print ("Confusion matrix :", metrics.confusion_matrix(y, model.predict(x)))
print ("classification report :", metrics.classification_report(y, model.predict(x)))

3-30　曲線下面積

In [ ]:
# 偽陽性率と真陽性率の決定
fpr, tpr, _ = metrics.roc_curve(y, model.predict_proba(x)[:,1])

# AUCの計算
roc_auc = metrics.auc(fpr, tpr)
print ('ROC AUC: %0.2f' % roc_auc)

# 特定のクラスに対するROC曲線のプロット
plt.plot()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


3-31. 直線フィッティングの複雑さの調節

In [ ]:
# デフォルトのc値でロジスティック回帰モデルをインスタンス化しXとyで適合させる
model = LogisticRegression()
model = model.fit(x, y)

# 学習データセットでの精度確認
print ("C = 1 (default), Accuracy :", metrics.accuracy_score(y, model.predict(x)))

# c = 10のロジスティック回帰モデルをインスタンス化しXとyで適合させる
model1 = LogisticRegression(C=10)
model1 = model1.fit(x, y)

# トレーニングセットでの精度確認
print ("C = 10, Accuracy :", metrics.accuracy_score(y, model1.predict(x)))

# c = 100のロジスティック回帰モデルをインスタンス化しXとyで適合させる
model2 = LogisticRegression(C=100)
model2 = model2.fit(x, y)

# トレーニングセットでの精度確認
print ("C = 100, Accuracy :", metrics.accuracy_score(y, model2.predict(x)))

# c = 1000のロジスティック回帰モデルをインスタンス化しXとyで適合させる
model3 = LogisticRegression(C=1000)
model3 = model3.fit(x, y)

# 学習データセットでの精度確認
print ("C = 1000, Accuracy :", metrics.accuracy_score(y, model3.predict(x)))

# フィッティングした線をプロット
plt.scatter(df.Hours_Studied, y,  color='black', label='Result')
plt.yticks([0.0, 0.5, 1.0])
plt.plot(df.Hours_Studied, model.predict_proba(x)[:,1], color='gray', linewidth=2, label='C=1.0')
plt.plot(df.Hours_Studied, model1.predict_proba(x)[:,1], color='blue', linewidth=2,label='C=10')
plt.plot(df.Hours_Studied, model2.predict_proba(x)[:,1], color='green', linewidth=2,label='C=100')
plt.plot(df.Hours_Studied, model3.predict_proba(x)[:,1], color='red', linewidth=2,label='C=1000')
plt.legend(loc='lower right') # 凡例の位置
plt.title('Hours Studied vs Result')
plt.ylabel('Result')
plt.xlabel('Hours_Studied')
plt.show()


3-32. 過剰適合，正しい適合，過小適合

In [ ]:
import pandas as pd

data = pd.read_csv('Data/LR_NonLinear.csv')
pos = data['class'] == 1
neg = data['class'] == 0
x1 = data['x1']
x2 = data['x2']

# 2つの変数の間の散布図を作成
def draw_plot():
    plt.figure(figsize=(6, 6))
    plt.scatter(np.extract(pos, x1), np.extract(pos, x2), c='b', marker='s', label='pos')
    plt.scatter(np.extract(neg, x1), np.extract(neg, x2), c='r', marker='o', label='neg')   
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.legend()

# 独立変数の高次多項式の作成
order_no = 6

# 変数1と2をその高次多項式に対応させる
def map_features(variable_1, variable_2, order=order_no):
    assert order >= 1
    def iter():
        for i in range(1, order + 1):
            for j in range(i + 1):
                yield np.power(variable_1, i - j) * np.power(variable_2, j)
    return np.vstack(iter())
out = map_features(data['x1'], data['x2'], order=order_no)
X = out.transpose()
y = data['class']

# データを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# 分類直線を描画するための関数
def draw_boundary(classifier):
    dim = np.linspace(-0.8, 1.1, 100)
    dx, dy = np.meshgrid(dim, dim)
    v = map_features(dx.flatten(), dy.flatten(), order=order_no)
    z = (np.dot(classifier.coef_, v) + classifier.intercept_).reshape(100, 100)
    plt.contour(dx, dy, z, levels=[0], colors=['r'])

# # c = 0.01として適合させる
clf = LogisticRegression(C=0.01).fit(X_train, y_train)
print ('Train Accuracy for C=0.01: ', clf.score(X_train, y_train))
print ('Test Accuracy for C=0.01: ', clf.score(X_test, y_test))
draw_plot()
plt.title('Fitting with C=0.01')
draw_boundary(clf)
plt.legend()

# # c = 1として適合させる
clf = LogisticRegression(C=1).fit(X_train, y_train)
print ('Train Accuracy for C=1: ', clf.score(X_train, y_train))
print ('Test Accuracy for C=1: ', clf.score(X_test, y_test))
draw_plot()
plt.title('Fitting with C=1')
draw_boundary(clf)
plt.legend()

# # c = 10000として適合させる
clf = LogisticRegression(C=10000).fit(X_train, y_train)
print ('Train Accuracy for C=10000: ', clf.score(X_train, y_train))
print ('Test Accuracy for C=10000: ', clf.score(X_test, y_test))
draw_plot()
plt.title('Fitting with C=10000')
draw_boundary(clf)
plt.legend()


3-33. データの読み込み

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd
iris = datasets.load_iris()
X = iris.data
y = iris.target
print('Class labels:', np.unique(y))


3-34. 正規化

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

3-35. 訓練データとテストデータの分割

In [ ]:
# 訓練データとテストデータの分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

3-36.  ロジスティック回帰モデルの学習とその評価

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# L1正規化で良い結果を求める
lr = LogisticRegression(penalty='l1', C=10, random_state=0,solver='liblinear')
lr.fit(X_train, y_train)

# 評価指標の生成
print("Train - Accuracy :", metrics.accuracy_score(y_train, lr.predict(X_train)))
print("Train - Confusion matrix :",metrics.confusion_matrix(y_train, lr.predict(X_train)))
print("Train - classification report :", metrics.classification_report(y_train, lr.predict(X_train)))
print("Test - Accuracy :", metrics.accuracy_score(y_test, lr.predict(X_test)))
print("Test - Confusion matrix :",metrics.confusion_matrix(y_test, lr.predict(X_test)))
print("Test - classification report :", metrics.classification_report(y_test, lr.predict(X_test)))


3-37. 一般化線形モデル

In [ ]:
df = pd.read_csv('Data/Grade_Set_1.csv')

print('####### Linear Regression Model ########')

# 線形回帰モデルの作成
lr = lm.LinearRegression()
x= df.Hours_Studied[:, np.newaxis] # 独立変数
y= df.Test_Grade.values            # 従属変数

# 訓練データセットを使ってモデルをトレーニング
lr.fit(x, y)
print ("Intercept: ", lr.intercept_)
print ("Coefficient: ", lr.coef_)
print('\n####### Generalized Linear Model ########')
import statsmodels.api as sm

# GLMを実行するためにはx変数に切片定数を追加する必要がある
x = sm.add_constant(x, prepend=False)

# デフォルトのリンク関数でガウスファミリーモデルをインスタンス化
model = sm.GLM(y, x, family = sm.families.Gaussian())
model = model.fit()
print (model.summary())


3-38. 決定木モデル

In [ ]:
# pydotをpip install しておく
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn import tree
iris = datasets.load_iris()
# X = iris.data[:, [2, 3]]
X = iris.data
y = iris.target
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

# 学習データとテストデータに分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
clf = tree.DecisionTreeClassifier(criterion = 'entropy', random_state=0)
clf.fit(X_train, y_train)

# 評価指標の生成
print("Train - Accuracy :", metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Train - Confusion matrix :",metrics.confusion_matrix(y_train, clf.predict(X_train)))
print("Train - classification report :", metrics.classification_report(y_train, clf.predict(X_train)))
print("Test - Accuracy :", metrics.accuracy_score(y_test, clf.predict(X_test)))
print("Test - Confusion matrix :",metrics.confusion_matrix(y_test, clf.predict(X_test)))
print("Test - classification report :", metrics.classification_report
(y_test, clf.predict(X_test)))
tree.export_graphviz(clf, out_file='tree.dot')

from six import StringIO
import pydot
out_data = StringIO()
tree.export_graphviz(clf, out_file=out_data,
                    feature_names=iris.feature_names,
                    class_names=clf.classes_.astype(int).astype(str),
                    filled=True, rounded=True,
                    special_characters=True,
                    node_ids=1,)
graph = pydot.graph_from_dot_data(out_data.getvalue())
# graph[0].write_pdf("iris.pdf")  # save to pdf


3-39. サポートベクターマシン

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import metrics
iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
print('Class labels:', np.unique(y))
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

# 学習データととリーニングデータの分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1.0, random_state=0)
clf.fit(X_train, y_train)

# 評価指標の生成
print("Train - Accuracy :", metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Train - Confusion matrix :",metrics.confusion_matrix(y_train, clf.predict(X_train)))
print("Train - classification report :", metrics.classification_report(y_train, clf.predict(X_train)))
print("Test - Accuracy :", metrics.accuracy_score(y_test, clf.predict(X_test)))
print("Test - Confusion matrix :", metrics.confusion_matrix(y_test, clf.predict(X_test)))
print("Test - classification report :", metrics.classification_report
(y_test, clf.predict(X_test)))

3-40.   SVMの決定境界線の設定

In [ ]:
#sklearnのmake_classification関数を使ってテストデータを作ってみよう
from sklearn.datasets import make_classification
from matplotlib.colors import ListedColormap

X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, weights=[.5, .5], random_state=0)

# 単純なロジスティック回帰モデルの構築
clf = SVC(kernel='linear', random_state=0)
clf.fit(X, y)

# 分離するための超平面を作成
w = clf.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(-5, 5)
yy = a * xx - (clf.intercept_[0]) / w[1]

# サポートベクタルを通過する超平面をプロット
b = clf.support_vectors_[0]
yy_down = a * xx + (b[1] - a * b[0])
b = clf.support_vectors_[-1]
yy_up = a * xx + (b[1] - a * b[0])


# 決定境界をプロットする
def plot_decision_regions(X, y, classifier):
    
    h = .02  # メッシュのステップサイズを設定
    # カラーマップを作成
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 決定境界のプロット
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, h),
                           np.arange(x2_min, x2_max, h))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, color=cmap(idx),
                    marker=markers[idx], label=cl)

plot_decision_regions(X, y, classifier=clf)

# 直線，点，平面に最も近いベクトルをプロットする
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=80, facecolors='none')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

3-41. ｋ近傍法

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
clf.fit(X_train, y_train)

# 評価指標の作成
print("Train - Accuracy :", metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Train - Confusion matrix :",metrics.confusion_matrix(y_train, clf.predict(X_train)))
print("Train - classification report :", metrics.classification_report(y_train, clf.predict(X_train)))
print("Test - Accuracy :", metrics.accuracy_score(y_test, clf.predict(X_test)))
print("Test - Confusion matrix :", metrics.confusion_matrix(y_test, clf.predict(X_test)))
print("Test - classification report :", metrics.classification_report(y_test, clf.predict(X_test)))


3-42. 時系列の分解

In [ ]:
# データソース: O.D. Anderson (1976), in file: data/anderson14,  X社の月次売上高 Jan '65 – May '71 C.
df = pd.read_csv('Data/TS.csv')
ts = pd.Series(list(df['Sales']), index=pd.to_datetime(df['Month'],format='%Y-%m'))
from statsmodels.tsa.seasonal import seasonal_decompose

# 対数変換
ts_log = np.log(ts)
ts_log.dropna(inplace=True)

decomposition = seasonal_decompose(ts)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
plt.subplot(411)
plt.plot(ts_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
from statsmodels.tsa.stattools import adfuller

s_test = adfuller(ts_log, autolag='AIC')
print ("Log transform stationary check p value: ", s_test[1])
# 最初の差分を取る
ts_log_diff = ts_log - ts_log.shift()
ts_log_diff.dropna(inplace=True)
s_test = adfuller(ts_log_diff, autolag='AIC')
print ("First order difference stationary check p value: ", s_test[1] )
# 移動平均で線を滑らかにする
moving_avg = ts_log.rolling(12).mean()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10,3))
ax1.set_title('First order difference')
ax1.tick_params(axis='x', labelsize=7)
ax1.tick_params(axis='y', labelsize=7)
ax1.plot(ts_log_diff)
ax2.plot(ts_log)
ax2.set_title('Log vs Moving AVg')
ax2.tick_params(axis='x', labelsize=7)
ax2.tick_params(axis='y', labelsize=7)
ax2.plot(moving_avg, color='red')
plt.tight_layout()


3-44. 自己相関検定

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10,3))
# ACFチャート
fig = sm.graphics.tsa.plot_acf(ts_log_diff.values.squeeze(), lags=20, ax=ax1)
# 95%信頼区間線の描画
ax1.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
ax1.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
ax1.set_xlabel('Lags')
# PACFチャート
fig = sm.graphics.tsa.plot_pacf(ts_log_diff, lags=20, ax=ax2)
# 95%信頼区間線の描画
ax2.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
ax2.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
ax2.set_xlabel('Lags')


3-45.  ARIMAモデルの構築と評価

In [ ]:
# モデルの構築
model = sm.tsa.ARIMA(ts_log, order=(2,0,2))
results_ARIMA = model.fit(disp=-1)
ts_predict = results_ARIMA.predict()

# モデルの評価
print("AIC: ", results_ARIMA.aic)
print("BIC: ", results_ARIMA.bic)
print("Mean Absolute Error: ", mean_absolute_error(ts_log.values, ts_predict.values))
print("Root Mean Squared Error: ", np.sqrt(mean_squared_error(ts_log.values, ts_predict.values)))

# 自己相関の確認
print("Durbin-Watson statistic :", sm.stats.durbin_watson(results_ARIMA.resid.values))

3-46. ARIMAモデルの構築とpを3に増大させた場合の評価


In [ ]:
model = sm.tsa.ARIMA(ts_log, order=(3,0,2))
results_ARIMA = model.fit(disp=-1)
ts_predict = results_ARIMA.predict()

plt.title('ARIMA Prediction - order(3,0,2)')
plt.plot(ts_log, label='Actual')
plt.plot(ts_predict, 'r--', label='Precicted')
plt.xlabel('Year-Month')
plt.ylabel('Sales')
plt.legend(loc='best')

print("AIC; ", results_ARIMA.aic)
print("BIC: ", results_ARIMA.bic)

print("Mean Absolute Error: ",
      mean_absolute_error(ts_log.values, ts_predict.values))
print("RootMean Squared Error: ",
      np.sqrt(mean_squared_error(ts_log.values, ts_predict.values)))

#自己相関の確認
print("Durbin-Watson statistic: ",
      sm.stats.durbin_watson(results_ARIMA.resid.values))

3-47.  一次差分とARIMA

In [ ]:
model = sm.tsa.ARIMA(ts_log, order=(3,1,2))
results_ARIMA = model.fit(disp=-1)
ts_predict = results_ARIMA.predict()

# 差分を補うための補正
predictions_ARIMA_diff = pd.Series(ts_predict, copy=True)
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_log = pd.Series(ts_log.iloc[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
plt.title('ARIMA Prediction - order(3,1,2)')
plt.plot(ts_log, label='Actual')
plt.plot(predictions_ARIMA_log, 'r--', label='Predicted')
plt.xlabel('Year-Month')
plt.ylabel('Sales')
plt.legend(loc='best')
print("AIC: ", results_ARIMA.aic)
print("BIC: ", results_ARIMA.bic)
print("Mean Absolute Error: ", mean_absolute_error(ts_log_diff.values, ts_predict.values))
print("Root Mean Squared Error: ", np.sqrt(mean_squared_error(ts_log_diff.values, ts_predict.values)))

# 自己相関の確認
print("Durbin-Watson statistic :", sm.stats.durbin_watson(results_ARIMA.resid.values))


3-48. ARIMAの予測関数

In [ ]:
# 最終モデル
model = sm.tsa.ARIMA(ts_log, order=(3,0,2))
results_ARIMA = model.fit(disp=-1)
# 将来値の予測
ts_predict = results_ARIMA.predict('1971-06-01', '1972-05-01')
plt.title('ARIMA Future Value Prediction - order(3,1,2)')
plt.plot(ts_log, label='Actual')
plt.plot(ts_predict, 'r--', label='Predicted')
plt.xlabel('Year-Month')
plt.ylabel('Sales')
plt.legend(loc='best')

3-49. k-meansクラスタリング

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
iris = datasets.load_iris()

# データフレームの変換
iris = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['species'])

# カラム名からスペースを削除
iris.columns = iris.columns.str.replace(' ','')
iris.head()
X = iris.iloc[:,:3]  # 独立変数
y = iris.species   # 従属変数
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

# K Meansクラスター
model = KMeans(n_clusters=3, random_state=11)
model.fit(X)
print (model.labels_)

3-50.  k-means クラスタリングの精度

In [ ]:
# 教師なしなのでラベルが割り当てられている
# 実際のラベルと一致しないので，すべての1を0に，0を1に変換する
# 結果として2が最適に見える
iris['pred_species'] =  np.choose(model.labels_, [1, 0, 2]).astype(np.int64)
print ("Accuracy :", metrics.accuracy_score(iris.species, iris.pred_species))
print ("Classification report :", metrics.classification_report(iris.species, iris.pred_species))

# プロットのサイズを設定
plt.figure(figsize=(10,7))

# 赤，緑，青のカラーマップの作成
cmap = ListedColormap(['r', 'g', 'b'])

# セパルをプロット
plt.subplot(2, 2, 1)
plt.scatter(iris['sepallength(cm)'], iris['sepalwidth(cm)'], c=cmap(iris.species), marker='o', s=50)
plt.xlabel('sepallength(cm)')
plt.ylabel('sepalwidth(cm)')
plt.title('Sepal (Actual)')
plt.subplot(2, 2, 2)
plt.scatter(iris['sepallength(cm)'], iris['sepalwidth(cm)'], c=cmap(iris.pred_species), marker='o', s=50)
plt.xlabel('sepallength(cm)')
plt.ylabel('sepalwidth(cm)')
plt.title('Sepal (Predicted)')
plt.subplot(2, 2, 3)
plt.scatter(iris['petallength(cm)'], iris['petalwidth(cm)'], c=cmap(iris.species),marker='o', s=50)
plt.xlabel('petallength(cm)')
plt.ylabel('petalwidth(cm)')
plt.title('Petal (Actual)')
plt.subplot(2, 2, 4)
plt.scatter(iris['petallength(cm)'], iris['petalwidth(cm)'], c=cmap(iris.pred_species),marker='o', s=50)
plt.xlabel('petallength(cm)')
plt.ylabel('petalwidth(cm)')
plt.title('Petal (Predicted)')
plt.tight_layout()

3-51. Elbow法

In [ ]:
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans
K = range(1,10)
KM = [KMeans(n_clusters=k).fit(X) for k in K]
centroids = [k.cluster_centers_ for k in KM]
D_k = [cdist(X, cent, 'euclidean') for cent in centroids]
cIdx = [np.argmin(D,axis=1) for D in D_k]
dist = [np.min(D,axis=1) for D in D_k]
avgWithinSS = [sum(d)/X.shape[0] for d in dist]

# 正方形の中の合計
wcss = [sum(d**2) for d in dist]
tss = sum(pdist(X)**2)/X.shape[0]
bss = tss-wcss
varExplained = bss/tss*100
kIdx = 10-1

### プロット ###
kIdx = 2
# elbowカーブ
# プロットのサイズを設定
plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
plt.plot(K, avgWithinSS, 'b*-')
plt.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12,
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elbow for KMeans clustering')
plt.subplot(1, 2, 2)
plt.plot(K, varExplained, 'b*-')
plt.plot(K[kIdx], varExplained[kIdx], marker='o', markersize=12,
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Percentage of variance explained')
plt.title('Elbow for KMeans clustering')
plt.tight_layout()


3-52.  シルエット分析

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from matplotlib import cm
score = []
for n_clusters in range(2,10):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    score.append(silhouette_score(X, labels, metric='euclidean'))

# プロットのサイズを設定
plt.figure(figsize=(10,4))
plt.subplot(1, 2, 1)
plt.plot(score)
plt.grid(True)
plt.ylabel("Silouette Score")
plt.xlabel("k")
plt.title("Silouette for K-means")

# n_clustersの値とランダム生成器でクラスターを初期化する
model = KMeans(n_clusters=3, init='k-means++', n_init=10, random_state=0)
model.fit_predict(X)
cluster_labels = np.unique(model.labels_)
n_clusters = cluster_labels.shape[0]

# 各サンプルのシルエット・スコアを計算する
silhouette_vals = silhouette_samples(X, model.labels_)
plt.subplot(1, 2, 2)

# colormapのスペクトル値を取得
cmap = cm.get_cmap("Spectral")
y_lower, y_upper = 0,0
yticks = []
for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[cluster_labels]
    c_silhouette_vals.sort()
    y_upper += len(c_silhouette_vals)
    color = cmap(float(i) / n_clusters)
    plt.barh(range(y_lower, y_upper), c_silhouette_vals, facecolor=color, edgecolor=color, alpha=0.7)
    yticks.append((y_lower + y_upper) / 2)
    y_lower += len(c_silhouette_vals)
    
silhouette_avg = np.mean(silhouette_vals)
plt.yticks(yticks, cluster_labels+1)

# 縦軸はすべての値における
plt.axvline(x=silhouette_avg, color="red", linestyle="--")
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')
plt.title("Silouette for K-means")
plt.show()


3-53.  階層型クラスタリング

In [ ]:
from sklearn.cluster import AgglomerativeClustering
# 凝集性クラスター
model = AgglomerativeClustering(n_clusters=3)
# リスト3-49でインポートした虹彩データセットにモデルを当てはめてみよう
model.fit(X)
print(model.labels_)
iris['pred_species'] =  model.labels_
print("Accuracy :", metrics.accuracy_score(iris.species, iris.pred_species))
print("Classification report :", metrics.classification_report(iris.species, iris.pred_species))


3-54. 階層クラスタリング

In [ ]:
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage
from scipy.spatial.distance import pdist
# 連結行列の生成
Z = linkage(X, 'ward')
c, coph_dists = cophenet(Z, pdist(X))
# 系統樹の計算
plt.figure(figsize=(25, 10))
plt.title('Agglomerative Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # X軸のラベルを回転
    leaf_font_size=8.,  # X軸のラベルのフォントサイズを変更
)
plt.tight_layout()


3-55. 主成分分析

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
iris = datasets.load_iris()
X = iris.data

# データの標準化
X_std = StandardScaler().fit_transform(X)

# 共分散行列の作成
cov_mat = np.cov(X_std.T)
print('Covariance matrix \n%s' %cov_mat)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

# 固有値を降順
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len
(eig_vals))]
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print("Cummulative Variance Explained", cum_var_exp)
plt.figure(figsize=(6, 4))
plt.bar(range(4), var_exp, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(4), cum_var_exp, where='mid', label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()


3-56. PCAの可視化

In [ ]:
# ソース: http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html#
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
iris = datasets.load_iris()
Y = iris.target

# ディメンションの相互作用をより深く理解する
# 最初の3つのPCA結果をプロット
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=110)
X_reduced = PCA(n_components=3).fit_transform(iris.data)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=Y, cmap=plt.cm.Paired)
ax.set_title("First three PCA directions")
ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])
plt.show()
